In [12]:
# import os
# path = r"C:\Users\32005\Desktop\手写数字\9"

# files = os.listdir(path)#原来文件夹的路径
# i = 0

# for file in files:
#     original = r"C:\Users\32005\Desktop\手写数字\9" + os.sep + files[i]
#     #修改后放置图片的路径 F:/ns，也可将 img_ 换成其他标注
#     new = r"C:\Users\32005\Desktop\手写数字\9" + os.sep + "j" + str(i + 1) + "_label_9.jpg"
#     os.rename(original, new)
#     i += 1

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#数据预处理" data-toc-modified-id="数据预处理-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>数据预处理</a></span><ul class="toc-item"><li><span><a href="#opencv读取图片文件" data-toc-modified-id="opencv读取图片文件-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>opencv读取图片文件</a></span></li></ul></li><li><span><a href="#建立模型" data-toc-modified-id="建立模型-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>建立模型</a></span></li><li><span><a href="#进行训练" data-toc-modified-id="进行训练-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>进行训练</a></span></li><li><span><a href="#模型准确性" data-toc-modified-id="模型准确性-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>模型准确性</a></span></li><li><span><a href="#进行预测" data-toc-modified-id="进行预测-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>进行预测</a></span></li></ul></div>

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 支持多行输出
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all' #默认为'last'

#   数据预处理

## opencv读取图片文件

In [15]:
import cv2
import numpy as np
import os

df = pd.read_csv('label.txt',header=None, sep=' ',index_col=0,names=['filename','num'])
label = df.to_dict('index')

baseDir = "imgdata"  # 读取文件的路径
filenames = os.listdir(baseDir)
imgdata = cv2.imread('imgdata/' + list(label.keys())[0], flags=1)
imgdata = cv2.resize(imgdata, (64, 64))
imgdata = imgdata.reshape(1,64,64,3)

#定义标签列表
y_train = []
for i in label:
    y_train.append(label[i].get('num'))
    if i != list(label.keys())[0]:
        imgfile = 'imgdata/' + i
        tmp_img = cv2.imread(imgfile, flags=1)
        tmp_img = cv2.resize(tmp_img, (64, 64))
        tmp_img = tmp_img.reshape(1,64,64,3)
        imgdata = np.concatenate((imgdata, tmp_img), axis=0)

X_train = imgdata
X_train_normalize = X_train.astype('float32') / 255.0

y_train = np.array(y_train)

In [16]:
X_train.shape
y_train.shape

(100, 64, 64, 3)

(100,)

In [17]:
index = [i for i in range(len(X_train))]
np.random.shuffle(index)
X_train = X_train[index]
y_train = y_train[index]

# 建立模型

In [18]:
import tensorflow.keras as keras
model = keras.models.Sequential()

#卷积层1
model.add(keras.layers.Conv2D(
                 filters=8,    
                 kernel_size=(3,3),
                 input_shape=(64,64,3),            
                 activation='relu')) # strides=(1, 1)是默认值
#池化层1
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

#卷积层2
model.add(keras.layers.Conv2D(
                 filters=16,
                 kernel_size=(3,3),
                 activation='relu'))
#池化层2
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

#卷积层3
model.add(keras.layers.Conv2D(
                 filters=32,
                 kernel_size=(3,3),
                 activation='relu'))
#池化层3
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

#展平层
model.add(keras.layers.Flatten())
#全连接层1
model.add(keras.layers.Dense(128, activation='relu'))
#Dropout
model.add(keras.layers.Dropout(0.5))
#全连接层2
model.add(keras.layers.Dense(1,activation='relu'))

model.compile(loss='mse',
              optimizer='adam',
              metrics=['mse'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 8)         224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

# 进行训练

In [19]:
train_history = model.fit(
                        x=X_train_normalize, 
                        y=y_train,
                        validation_split=0.2, 
                        epochs=100, 
                        batch_size=20,
                        verbose=1
                        )

Epoch 1/100
4/4 [==============================] - 0s 35ms/step - loss: 16.8760 - mse: 16.8760 - val_loss: 12.6424 - val_mse: 12.6424
Epoch 2/100
4/4 [==============================] - 0s 12ms/step - loss: 11.2971 - mse: 11.2971 - val_loss: 10.6667 - val_mse: 10.6667
Epoch 3/100
4/4 [==============================] - 0s 12ms/step - loss: 9.8854 - mse: 9.8854 - val_loss: 11.5882 - val_mse: 11.5882
Epoch 4/100
4/4 [==============================] - 0s 11ms/step - loss: 8.8910 - mse: 8.8910 - val_loss: 13.5784 - val_mse: 13.5784
Epoch 5/100
4/4 [==============================] - 0s 11ms/step - loss: 9.2110 - mse: 9.2110 - val_loss: 11.9056 - val_mse: 11.9056
Epoch 6/100
4/4 [==============================] - 0s 12ms/step - loss: 8.3364 - mse: 8.3364 - val_loss: 10.4180 - val_mse: 10.4180
Epoch 7/100
4/4 [==============================] - 0s 12ms/step - loss: 9.4427 - mse: 9.4427 - val_loss: 10.4127 - val_mse: 10.4127
Epoch 8/100
4/4 [==============================] - 0s 11ms/step - loss: 

Epoch 63/100
4/4 [==============================] - 0s 13ms/step - loss: 1.6405 - mse: 1.6405 - val_loss: 10.9077 - val_mse: 10.9077
Epoch 64/100
4/4 [==============================] - 0s 12ms/step - loss: 1.8805 - mse: 1.8805 - val_loss: 10.1204 - val_mse: 10.1204
Epoch 65/100
4/4 [==============================] - 0s 12ms/step - loss: 1.6883 - mse: 1.6883 - val_loss: 9.7582 - val_mse: 9.7582
Epoch 66/100
4/4 [==============================] - 0s 14ms/step - loss: 1.9534 - mse: 1.9534 - val_loss: 11.1845 - val_mse: 11.1845
Epoch 67/100
4/4 [==============================] - 0s 13ms/step - loss: 2.0351 - mse: 2.0351 - val_loss: 9.6378 - val_mse: 9.6378
Epoch 68/100
4/4 [==============================] - 0s 13ms/step - loss: 2.3255 - mse: 2.3255 - val_loss: 10.7589 - val_mse: 10.7589
Epoch 69/100
4/4 [==============================] - 0s 14ms/step - loss: 1.4776 - mse: 1.4776 - val_loss: 10.1656 - val_mse: 10.1656
Epoch 70/100
4/4 [==============================] - 0s 13ms/step - loss: 

# 模型准确性

In [21]:
scores = model.evaluate(X_train_normalize, y_train)
scores[1]

4/4 [==============================] - 0s 3ms/step - loss: 2.4492 - mse: 2.4492


2.449223518371582

# 进行预测

In [31]:
import cv2
import numpy as np
imgdata_src = cv2.imread('0/j8_label_0.jpg', flags=1)
imgdata_resized = cv2.resize(imgdata_src, (64, 64))
imgdata = imgdata_resized.reshape(64,64,3)
imgdata_4D = imgdata.reshape(1,64,64,3)
imgdata_4D = imgdata_4D/255


result_arry = model.predict(imgdata_4D)[0]
result = round(result_arry[0])

print(result_arry)
print("预测的数字是：" + str(result))

[5.758824]
预测的数字是：6


In [23]:
import cv2
import numpy as np
imgdata_src = cv2.imread('5/j1_label_5.jpg', flags=1)
imgdata_resized = cv2.resize(imgdata_src, (64, 64))
imgdata = imgdata_resized.reshape(64,64,3)
imgdata_4D = imgdata.reshape(1,64,64,3)
imgdata_4D = imgdata_4D/255


result_arry = model.predict(imgdata_4D)[0]
result = round(result_arry[0])

print(result_arry)
print("预测的数字是：" + str(result))

[2.6901698]
预测的数字是：3


In [35]:
import cv2
import numpy as np
imgdata_src = cv2.imread('7/j_label_7.jpg', flags=1)
imgdata_resized = cv2.resize(imgdata_src, (64, 64))
imgdata = imgdata_resized.reshape(64,64,3)
imgdata_4D = imgdata.reshape(1,64,64,3)
imgdata_4D = imgdata_4D/255


result_arry = model.predict(imgdata_4D)[0]
result = round(result_arry[0])

print(result_arry)
print("预测的数字是：" + str(result))

[3.4398963]
预测的数字是：3
